<a href="https://colab.research.google.com/github/GIMMI42PIASTRATO/AI-Chess-BOT/blob/main/AI_Chess_BOT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Installation of all the dependencies

In [27]:
!pip install kaggle
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [28]:
!kaggle datasets download arevel/chess-games
!unzip -qq /content/chess-games.zip

chess-games.zip: Skipping, found more recently modified local copy (use --force to force download)
replace chess_games.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [29]:
!pip install chess -q

#CODE

In [30]:
import chess
import numpy as np
import pandas as pd
from pandas.core.generic import gc
import re

##Mapping the chess board

In [31]:
letter_to_num = {'a': 0, 'b': 1, 'c': 2, 'd': 3, 'e': 4, 'f': 5, 'g': 6, 'h': 7}
num_to_letter = {0: 'a', 1: 'b', 2: 'c', 3: 'd', 4: 'e', 5: 'f', 6: 'g', 7: 'h'}

##Chess board to matrix

###Board to rappresentation
This function create a numpy tensor (3D numpyarray) which rapresent the board containing every position of every chess pice.
Every chess pice has is own mapping (Is a matrix representing the chessboard that maps the position of only one type of pieces)

In [32]:
def board_to_rep(board: chess.Board()):
  pieces = ['p', 'r', 'n', 'b', 'q', 'k'] #Initials of the names of the pieces
  layers = []
  for piece in pieces:
    layers.append(layer_to_rep(board, piece))
  board_rep = np.stack(layers)
  return board_rep

###Layer rappresentation

In [33]:
def layer_to_rep(board, piece):
  str_board = str(board)
  str_board = re.sub(f"[^{piece}{piece.upper()} \n]", ".", str_board)
  str_board = re.sub(f"{piece}", "1", str_board)
  str_board = re.sub(f"{piece.upper()}", "-1", str_board)
  str_board = re.sub("\.", "0", str_board)

  board_matrix = []
  for row in str_board.split("\n"):
    row = row.split()
    row = [int(x) for x in row]
    board_matrix.append(row)

  return np.array(board_matrix)

##Chess move to matrix

###Move to rappresentation

In [34]:
def move_to_rep(move, board: chess.Board()):
  board.push_san(move).uci()
  move = str(board.pop())

  from_output_layer = np.zeros((8, 8))
  from_row = 8 - int(move[1])
  from_column = letter_to_num[move[0]]
  from_output_layer[from_row][from_column] = 1

  to_output_layer = np.zeros((8, 8))
  to_row = 8 - int(move[3])
  to_column = letter_to_num[move[2]]
  to_output_layer[to_row][to_column]

  return np.stack((from_output_layer, to_output_layer))

##Create move list from dataset move list


In [35]:
def create_move_list(string_move_list):
  return re.sub("\d*\. ", '', string_move_list).split()[:-1]

#Importing the raw data

In [36]:
chess_data_raw = pd.read_csv("/content/chess_games.csv", usecols=["AN", "WhiteElo", "BlackElo"])

In [37]:
chess_data = chess_data_raw[(chess_data_raw["WhiteElo"] >= 1900) & (chess_data_raw["BlackElo"] >= 1900)]
del chess_data_raw
gc.collect()
chess_data = chess_data[["AN"]]
chess_data = chess_data[~chess_data["AN"].str.contains("{")]
chess_data = chess_data[chess_data["AN"].str.len() > 20]
print(chess_data.shape)

(1017420, 1)


In [38]:
chess_data.head()

,AN
4,1. e4 c5 2. Nf3 d6 3. d4 cxd4 4. Nxd4 Nf6 5. N...
7,1. d4 d5 2. Nf3 Nf6 3. Bf4 c6 4. e3 Bg4 5. Be2...
8,1. d4 Nf6 2. Bf4 e6 3. e3 d5 4. Nf3 h6 5. Bd3 ...
13,1. b4 e6 2. Bb2 d5 3. Nf3 Nf6 4. b5 Be7 5. e3 ...
16,1. e4 c6 2. Nf3 d5 3. Nc3 g6 4. d3 Bg7 5. Be2 ...
